In [1]:
import pandas as pd
import numpy as np

In [2]:
def DescribeDf(df):
    Columnas =df.columns
    Tipos=df.dtypes
    NumFilas=df.shape[0]
    NumColumnas=df.shape[1]
    print("Filas    = " +str(NumFilas))
    print("Columnas = " +str(NumColumnas))
    for i in range(NumColumnas):
        print('-'*50)
        print("Campo "+str(i+1)+'/'+str(NumColumnas)+" (Index "+str(i)+"): "+Columnas[i])
        print("")
        VC=df[Columnas[i]].value_counts()
        ValoresDistintos=VC.shape[0]
        if ValoresDistintos<=40:
            print(VC)
        elif ValoresDistintos==NumFilas:
            print("   >>> Posible clave principal <<<")
            print("")
            VC=df[Columnas[i]].value_counts(bins=10)
            print(VC)
        elif Tipos[i]=='int64' or Tipos[i]=='float64':
            print("ValoresDistintos="+str(NumFilas))
            print("")
            VC=df[Columnas[i]].value_counts(bins=20)
            print(VC)

In [3]:
def CreaDF(ruta,YearFolder,File):
    df=pd.read_csv(r'' + ruta + '\\' + YearFolder + '\\' + File,low_memory=False)
    print(df.shape)
    return df

In [4]:
def AnexaDF(df_total,ruta,YearFolder,File):
    print('-'*30)
    print(df_total.shape)
    df=pd.read_csv(r'' + ruta + '\\' + YearFolder + '\\' + File,low_memory=False)
    print(df.shape)
    df_total=pd.concat([df_total,df],ignore_index=True)
    print(df_total.shape)
    return df_total

In [5]:
def TraduceAunqueHayaNaNs(df,diccionario,CampoOrigen,CampoDestino):
    Filas=df.shape[0]
    FilasRellenas=0
    for i in range(Filas):
        entero=df.loc[i,CampoOrigen]
        if entero==entero:
            cadena=diccionario[entero]
            df.loc[i,CampoDestino]=cadena
            FilasRellenas=FilasRellenas+1
    print("Filas = "+str(Filas))
    print("FilasRellenas = "+str(FilasRellenas))
    Ratio=FilasRellenas/Filas
    print("Ratio[%] = "+str(100*Ratio))
    print("-"*10)
    print(df[CampoDestino].value_counts(1))

In [6]:
def Columnizar(df):
    
    Filas=df.shape[0]
    Columnas=df.shape[1]
    NombreColumna=df.columns
    Indice=df.index
    
    #print(Filas)
    #print(Columnas)
    #print(NombreColumna)
    #print(Indice)
    
    for i in range(Columnas):
        if i==0:
            col_tot=pd.DataFrame(df[NombreColumna[i]],index=Indice,columns=[NombreColumna[i]])
            col_tot['Cuenta']=NombreColumna[i]
            col_tot.rename(columns={col_tot.columns[0]:'Valor'}, inplace=True)
        else:
            col=pd.DataFrame(df[NombreColumna[i]],index=Indice,columns=[NombreColumna[i]])
            col['Cuenta']=NombreColumna[i]
            col.rename(columns={col.columns[0]:'Valor'}, inplace=True)
            col_tot=pd.concat([col_tot,col],axis=0)
    
    return col_tot

In [ ]:
def AgregaCalcula(df_detalle,WeightField,Field_Sample,AggField):
    
    df_detalle[Field_Sample+'_population_total']=df_detalle[WeightField]*df_detalle[Field_Sample]
    
    df_counts=df_detalle.groupby(AggField).agg(
        SumWeightField=pd.NamedAgg(column=WeightField, aggfunc=sum),
        SumFieldPopulationTotal=pd.NamedAgg(column=Field_Sample+'_population_total', aggfunc=sum)
        )
    
    df_counts.rename(columns={
        'SumWeightField':'Sum'+WeightField,
        'SumFieldPopulationTotal':'Sum'+Field_Sample+'_population_total'
    },inplace=True)
    
    df_counts[Field_Sample]=df_counts['Sum'+Field_Sample+'_population_total']/df_counts['Sum'+WeightField]
    
    return df_counts

In [ ]:
# create a list of our conditions
conditions = [
    (df_EPF_hog_gast_counts['EDADSP'] <= 25),
    (df_EPF_hog_gast_counts['EDADSP'] > 25) & (df_EPF_hog_gast_counts['EDADSP'] <= 35),
    (df_EPF_hog_gast_counts['EDADSP'] > 35) & (df_EPF_hog_gast_counts['EDADSP'] <= 45),
    (df_EPF_hog_gast_counts['EDADSP'] > 45) & (df_EPF_hog_gast_counts['EDADSP'] <= 55),
    (df_EPF_hog_gast_counts['EDADSP'] > 55) & (df_EPF_hog_gast_counts['EDADSP'] <= 65),
    (df_EPF_hog_gast_counts['EDADSP'] > 65)
    ]

# create a list of the values we want to assign for each condition
values = ['25 o menos', '25 a 35 ', '35 a 45', '45 a 55','55 a 65', '65 o mas']

# create a new column and use np.select to assign values to it using our lists as arguments
df_EPF_hog_gast_counts['RangoEDADSP'] = np.select(conditions, values)

In [ ]:
def WeigthedPercentile(df,WeightField,Field,Percentile,Verbosity=0):
    
    df=df.sort_values(Field)
    
    SumWeightField=df[WeightField].sum()
    if Verbosity>=1:
        print(SumWeightField)
    
    Rows=df.shape[0]
    if Verbosity>=1:
        print(str(Rows)+'\n')
    
    if Verbosity>=2:
        print('Row|Percentile|CummulativeWeigth|Value')
    for i in range(Rows):
        
        Weight=df[[WeightField,Field]].iloc[:i,0].sum()
        Ratio=(Weight*100)/SumWeightField
        PercentileValue=df[[WeightField,Field]].iloc[i,1]
        
        if Verbosity>=2:
            print(str(i)+'  | '+str(round(Ratio,2))+' | '+str(round(Weight,2))+' | '+str(PercentileValue))
        
        if Ratio>=Percentile:
            return PercentileValue